In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import date

url="http://web.mta.info/developers/turnstile.html"


# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")

<b>We will start by parsing the html using BeautifulSoup to pull all links from the MTA website. We will pull all links containing the data by date.<b>

In [2]:
list_of_links = []
for link in soup.find_all("a"):
    if '.txt' == "{}".format(link.get("href"))[-4:]:
        if "d" == "{}".format(link.get("href"))[0]:
            list_of_links.append(link.get("href"))
list_of_links

['data/nyct/turnstile/turnstile_210306.txt',
 'data/nyct/turnstile/turnstile_210227.txt',
 'data/nyct/turnstile/turnstile_210220.txt',
 'data/nyct/turnstile/turnstile_210213.txt',
 'data/nyct/turnstile/turnstile_210206.txt',
 'data/nyct/turnstile/turnstile_210130.txt',
 'data/nyct/turnstile/turnstile_210123.txt',
 'data/nyct/turnstile/turnstile_210116.txt',
 'data/nyct/turnstile/turnstile_210109.txt',
 'data/nyct/turnstile/turnstile_210102.txt',
 'data/nyct/turnstile/turnstile_201226.txt',
 'data/nyct/turnstile/turnstile_201219.txt',
 'data/nyct/turnstile/turnstile_201212.txt',
 'data/nyct/turnstile/turnstile_201205.txt',
 'data/nyct/turnstile/turnstile_201128.txt',
 'data/nyct/turnstile/turnstile_201121.txt',
 'data/nyct/turnstile/turnstile_201114.txt',
 'data/nyct/turnstile/turnstile_201107.txt',
 'data/nyct/turnstile/turnstile_201031.txt',
 'data/nyct/turnstile/turnstile_201024.txt',
 'data/nyct/turnstile/turnstile_201017.txt',
 'data/nyct/turnstile/turnstile_201010.txt',
 'data/nyc

<b> Noting that mapping updated on October 11, 2014, we will create two separate lists of dataframes to append data. Newer Data header is included in file, whereas older data requires the MTA mapping. <b>

In [ ]:
PrecursorFile = "http://web.mta.info/developers/"
NewerData = []
OlderData = []
Older_Data_Mapping = "C/A,UNIT,SCP,DATE1,TIME1,DESC1,ENTRIES1,EXITS1,DATE2,TIME2,DESC2,ENTRIES2,EXITS2,DATE3,TIME3,DESC3,ENTRIES3,EXITS3,DATE4,TIME4,DESC4,ENTRIES4,EXITS4,DATE5,TIME5,DESC5,ENTRIES5,EXITS5,DATE6,TIME6,DESC6,ENTRIES6,EXITS6,DATE7,TIME7,DESC7,ENTRIES7,EXITS7,DATE8,TIME8,DESC8,ENTRIES8,EXITS8".split(',')
for link in list_of_links:
    datafile = PrecursorFile + str(link)
    Date_Of_File = datafile.split("_")[1].split(".")[0]
    New_Date = pd.to_datetime(Date_Of_File,format='%y%m%d')
    if New_Date > date(2014,10,11):
        NewerData.append(pd.read_csv(datafile))
        print(New_Date)
    else: 
        OlderData.append(pd.read_csv(datafile,names=Older_Data_Mapping))

2021-03-06 00:00:00
2021-02-27 00:00:00
2021-02-20 00:00:00
2021-02-13 00:00:00
2021-02-06 00:00:00
2021-01-30 00:00:00
2021-01-23 00:00:00
2021-01-16 00:00:00
2021-01-09 00:00:00
2021-01-02 00:00:00
2020-12-26 00:00:00
2020-12-19 00:00:00
2020-12-12 00:00:00
2020-12-05 00:00:00
2020-11-28 00:00:00
2020-11-21 00:00:00
2020-11-14 00:00:00
2020-11-07 00:00:00
2020-10-31 00:00:00
2020-10-24 00:00:00
2020-10-17 00:00:00
2020-10-10 00:00:00
2020-10-03 00:00:00
2020-09-26 00:00:00
2020-09-19 00:00:00
2020-09-12 00:00:00
2020-09-05 00:00:00
2020-08-29 00:00:00
2020-08-22 00:00:00
2020-08-15 00:00:00
2020-08-08 00:00:00
2020-08-01 00:00:00
2020-07-25 00:00:00
2020-07-18 00:00:00
2020-07-11 00:00:00
2020-07-04 00:00:00
2020-06-27 00:00:00
2020-06-20 00:00:00
2020-06-13 00:00:00
2020-06-06 00:00:00
2020-05-30 00:00:00
2020-05-23 00:00:00
2020-05-16 00:00:00
2020-05-09 00:00:00
2020-05-02 00:00:00
2020-04-25 00:00:00
2020-04-18 00:00:00
2020-04-11 00:00:00
2020-04-04 00:00:00
2020-03-28 00:00:00


<b> We will begin with cleaning up the older data to meet requests. <b>

In [ ]:
NewerData_MTA_0302 = pd.concat(NewerData)
OlderData_MTA_0302 = pd.concat(OlderData)

info = OlderData_MTA_0302.iloc[:,0:3]
#Create DF with organized data by row. Note that df_3 must be added to df_2
Columns = ['C/A','UNIT','SCP','DATE','TIME','DESC','ENTRIES','EXITS']
New_List = []
for i in range(0,8): #Note dF follows a specific pattern, apply pattern (5*i+3), (5*i +8) to remove EntriesN/EntriesN columns, add to rows. 
    df_i = pd.concat([info, OlderData_MTA_0302.iloc[:,(5*i+3):(5*i + 8)]], axis=1)
    df_i.columns = Columns
    New_List.append(df_i)
OldData_Combined = pd.concat(New_List,axis=0)
OldData_Combined  = OldData_Combined.sort_values(by = ['C/A','UNIT','SCP','DATE','TIME'])

In [ ]:
OldData_Combined = pd.concat(New_List,axis=0)
OldData_Combined  = OldData_Combined.sort_values(by = ['C/A','UNIT','SCP','DATE','TIME'])

In [ ]:
OldData_Combined['DATETIME'] = pd.to_datetime((OldData_Combined['DATE'] + ' ' + OldData_Combined['TIME']), format='%m-%d-%y %H:%M:%S', errors = 'coerce')

In [ ]:
#Convert Date from String, convert Time from string. Create datetime field. 

OldData_Combined['DATE'] = pd.to_datetime(OldData_Combined['DATE'], errors = 'coerce')
OldData_Combined = OldData_Combined.dropna(subset=['DATE'])
OldData_Combined['TIME'] = pd.to_timedelta(OldData_Combined['TIME'], errors = 'coerce')
OldData_Combined = OldData_Combined.dropna(subset=['TIME'])


In [ ]:
OldData_Combined['ENTRIES'] = pd.to_numeric(OldData_Combined['ENTRIES'], errors = 'coerce')
OldData_Combined['EXITS'] = pd.to_numeric(OldData_Combined['EXITS'], errors = 'coerce')
OldData_Combined = OldData_Combined.dropna(subset=['ENTRIES'])
OldData_Combined = OldData_Combined.dropna(subset=['EXITS'])

In [ ]:
#Create Turnstile identification. Based on turnstile, calculate net difference between turnstile entries/exits. 

OldData_Combined['IDENTIFICATION'] = OldData_Combined['C/A'] + ':' + OldData_Combined['UNIT'] + ':' + OldData_Combined['SCP']
OldData_Combined  = OldData_Combined.sort_values(by = ['IDENTIFICATION','DATETIME'])

OldData_Combined['NET_ENTRIES'] = OldData_Combined.groupby(['IDENTIFICATION'])['ENTRIES'].transform(pd.Series.diff)
OldData_Combined['NET_EXITS'] = OldData_Combined.groupby(['IDENTIFICATION'])['EXITS'].transform(pd.Series.diff)


In [ ]:
#Note that Net Exits is a negative number. This does not make sense. We will make adjustments to the data. 

OldData_Combined[OldData_Combined.DATE == '02-01-13'].groupby('DATE')['NET_ENTRIES','NET_EXITS'].sum().round(2)

<b> Our logic will be the following: if Net Entries/Exits is less than 0, or greater than 20,000 (note that a 4 hour difference equates to 1.4 entries/exits per second), then set the value equal to the mean.<b>

In [ ]:
import numpy as np 

OldData_Combined['NET_ENTRIES'] = OldData_Combined.groupby(['IDENTIFICATION']).NET_ENTRIES.transform( lambda x: np.where((x<0)|(x>20000),x.mask((x<0)|(x>20000)).mean(),x))

In [ ]:
OldData_Combined['NET_EXITS'] = OldData_Combined.groupby(['IDENTIFICATION']).NET_EXITS.transform(lambda x: np.where((x<0)|(x>20000),x.mask((x<0)|(x>20000)).mean(),x))

<b> We will denote busyness as the sum of net entries + net exits. <b>

In [ ]:
OldData_Combined['BUSYNESS'] = OldData_Combined['NET_ENTRIES'] + OldData_Combined['NET_EXITS']

<b> We will join in Station data to map the 'old' dataset. <b>

In [ ]:
StationData = pd.read_excel('http://web.mta.info/developers/resources/nyct/turnstile/Remote-Booth-Station.xls')
StationData = StationData[['Booth', 'Station']]
StationData.rename(columns = {'Booth': 'C/A', 'Station': 'STATION'}, inplace = True)
StationData.head()

#Merge on ['C/A']
OldData_Combined = pd.merge(OldData_Combined, StationData, on = ['C/A'])


# DATA ANALYSIS QUESTIONS 

<b> Question 1: Which Station has the most number of Units? 
    We will make the assumption that this is referring to the Newer dataset, and measure which station has the most units from 2014 through 2020.<b>

In [ ]:
NewerData_MTA_0302.groupby(['STATION'],sort = False).nunique().UNIT.sort_values().tail(1)

<b> Question 1: Results show that 23rd street station has the most units. <b>

<b> Question 2:  What is the total number of entries & exits across the subway system for February 1, 2013? 
    
    We will use the adjusted dataset as the raw data produces a negative value for net exits. 

In [ ]:
OldData_Combined[OldData_Combined.DATE == '02-01-13'].groupby('DATE')['NET_ENTRIES','NET_EXITS'].sum().round(2)

<b> Question 2: On 2013-02-01, our results show	5,760,861 net entries and 4,465,555 net exits. (Entries/Exits must be whole numbers). <b>

<b> Let’s define the busy-ness as sum of entry & exit count. What station was the busiest on February 1, 2013? What turnstile was the busiest on that date? <b>


In [ ]:
OldData_Combined.head()

In [ ]:
OldData_Combined[OldData_Combined.DATE == '02-01-13'].groupby(['STATION_x']).sum().BUSYNESS.sort_values().tail(1)


In [ ]:
OldData_Combined[OldData_Combined.DATE == '02-01-13'].sort_values(by = 'BUSYNESS', ascending = False).head(1)['IDENTIFICATION']

<b> Results show that 34st - Penn Station is the busiest station. Intuitively, these results make sense given the high volume of commuters through Penn. While Penn Station was the busiest, N601:R319:00-00-00 was the busiest turnstile. <b>

<b> What stations have seen the most usage growth/decline in 2013?

We will approach this by calculating the percent difference relative to 2012 to define growth/decline.<b>

In [ ]:
OldData_Combined['YEAR'] = pd.DatetimeIndex(OldData_Combined['DATE']).year
OldData_Combined['MONTH'] = pd.DatetimeIndex(OldData_Combined['DATE']).month

#Calculate for 2013, calculate for 2012, calculate percent difference. 
AveragebyStation_2013 = OldData_Combined[OldData_Combined['YEAR'] == 2013].groupby(['STATION_x']).mean().BUSYNESS
AveragebyStation_2012 = OldData_Combined[OldData_Combined['YEAR'] == 2012].groupby(['STATION_x']).mean().BUSYNESS
Percent_Difference_1213 = ((AveragebyStation_2013 - AveragebyStation_2012) / AveragebyStation_2012)




In [ ]:
Percent_Difference_1213.sort_values(ascending = False).head(3)

In [ ]:
Percent_Difference_1213.sort_values(ascending = True).head(3)

<b> AQUEDUCT TRACK, 52 ST-LINCOLN , and BROADWAY/LAFAY show the highest % growth YoY (GROWTH). 
 
E TREMONT AVE, BEDFORD PARK BL, and BEACH 90 ST show the highest % decline YoY - These stations contain the most negative percent difference. <b> 

<b> What dates are the least busy? Could you identify days on which stations were not operating at full capacity or closed entirely? 
    
    Our approach will be to calculate the mean 'BUSYNESS' by date. Dates w/ lowest busyness should reflect closure/failure to operate at full capacity.


In [ ]:
OldData_Combined.groupby(['DATE']).mean().BUSYNESS.sort_values(ascending = True).head(4)

<b> Results show that 2011-08-28, 2012-10-30, 2012-10-29, 2012-10-31 had the lowest mean busyness. This is interesting because these are dates that the metro was completely closed: Aug 28, 2011 caused full closure due to Hurricane Irene. October 29 - October 31, 2012 showed full closure due to Hurricane Sandy. 
    
    These results make sense due to the liability/safety risk associated with the hurricane on these dates. <b>

<b> Bonus:  What hour is the busiest for station CANAL ST in Q1 2013? <b>

In [ ]:
OldData_Combined['HOUR'] = (OldData_Combined['TIME'] / np.timedelta64(1, 'h')).astype(int)
OldData_Combined[(OldData_Combined['YEAR'] == 2013) & (OldData_Combined['MONTH'] < 4) & (OldData_Combined['STATION_x'] == 'CANAL ST')].groupby(['HOUR']).mean().BUSYNESS.sort_values(ascending = False).head(4)


<b> Hour 20 is the busiest for Station Canal Street. <b>

# DATA VISUALIZATION QUESTIONS

In [ ]:
import matplotlib.pyplot as plt


df_q1_2013 = OldData_Combined[(OldData_Combined['YEAR'] == 2013) & (OldData_Combined['MONTH'] < 4)]
df_q1_2013_A = df_q1_2013.groupby(['DATE']).count().iloc[:,0]
df_q1_2013_A.head()

<b> Plot the daily row counts for data files in Q1 2013. <b>

In [ ]:
df_q1_2013_A.plot(linewidth=0.5)
plt.title('Q1 2013 Daily Row Count vs. Date', fontsize = 15,fontweight='bold')
plt.ylabel('Row Count', fontsize = 15, fontweight='bold')
plt.xlabel('Date', fontsize = 15, fontweight='bold')
plt.show()



<b> Plot the daily total number of entries & exits across the system for Q1 2013.<b> 


 

In [ ]:
q1_2013_daily_entry_exit= df_q1_2013.groupby(['DATE']).sum()[['NET_ENTRIES','NET_EXITS']]
q1_2013_daily_entry_exit.plot()
plt.style.use('ggplot')
plt.ylabel('Total Number', fontsize = 15)
plt.gcf().set_size_inches(20, 10, forward=True)
plt.xlabel('Date', fontsize = 15)
plt.title('2013 Q1 Entries + Exits', fontsize = 15, fontweight ='bold')
plt.show()

<b>Interesting that we observe weekly seasonality trend. Makes sense given that there is less commute(to and from work) on weekend. 

Number of exits is always lower than number of entries. this suggests that people potentially exit via emergency door rather than turnstile. <b>

<b> Plot the mean and standard deviation of the daily total number of entries & exits for each month in Q1 2013 for station 34 ST-PENN STA.<b> 

In [ ]:
q1_2013_mean_stdev = df_q1_2013[df_q1_2013['STATION_x'] == '34 ST-PENN STA'].groupby(['MONTH', 'DATE'])[['NET_ENTRIES','NET_EXITS']].sum()

Mean_StdDev = q1_2013_mean_stdev.groupby('MONTH')['NET_ENTRIES', 'NET_EXITS'].agg(['mean','std'])
Mean_StdDev

months = list([1,2,3])
mean_entries = Mean_StdDev['NET_ENTRIES', 'mean']
std_entries = Mean_StdDev['NET_ENTRIES', 'std']
mean_exits = Mean_StdDev['NET_EXITS', 'mean']
std_exits = Mean_StdDev['NET_EXITS', 'std']

Mean_StdDev.head()


plt.errorbar(months, mean_entries, yerr = std_entries,marker='.', linewidth = 2, ms = 30, mfc='b')
plt.xticks([1,2,3],['Jan','Feb','Mar'])
plt.xlabel('Month')
plt.ylabel('Total Number of Entries')
plt.title('Mean,Std.Dev of Entries', fontsize = 15, fontweight='bold')
plt.style.use('seaborn-talk')
plt.show()


plt.errorbar(months, mean_exits, yerr = std_exits,marker='.', linewidth = 2, ms = 30, mfc='b')
plt.xticks([1,2,3],['Jan','Feb','Mar'])
plt.xlabel('Month')
plt.ylabel('Total Number of Exits')
plt.title('Mean, Std.Dev of Exits', fontsize = 15, fontweight='bold')
plt.style.use('seaborn-talk')
plt.show()


<b> Plot 25/50/75 percentile of the daily total number of entries & exits for each month in Q1 2013 for station 34 ST-PENN STA.<b> 

In [ ]:
df_penn_daily = df_q1_2013[df_q1_2013['STATION_x'] == '34 ST-PENN STA'].groupby(['MONTH', 'DATE'])[['NET_ENTRIES','NET_EXITS']].sum().reset_index()

#Create function to prepare lists for percentile charts. 
def Graph_Setup():
    listOne = []
    listTwo = []
    for i in range(1, 4):
        listOne.append(df_penn_daily[df_penn_daily['MONTH'] == i]['NET_ENTRIES'])
        listTwo.append(df_penn_daily[df_penn_daily['MONTH'] == i]['NET_EXITS'])
    return [listOne, listTwo];

results = Graph_Setup()
listOfNetEntries = results[0]
listOfNetExits = results[1]


plt.subplot(1,2,1)
plt.boxplot(listOfNetEntries)
plt.title('2013 Q1 Boxplot: Daily Entries', fontsize = 15,fontweight='bold')
plt.xlabel('Month', fontsize = 15, fontweight='bold')
plt.ylabel('Entries', fontsize = 15, fontweight='bold')
plt.xticks([1,2,3],['Jan', 'Feb', 'March'], fontsize=15)
plt.show()



plt.subplot(1,2,2)
plt.boxplot(listOfNetExits)
plt.title('2013 Q1 Boxplot: Daily Exits', fontsize = 15,fontweight='bold')
plt.xlabel('Month', fontsize = 15, fontweight='bold')
plt.ylabel('Exits', fontsize = 15, fontweight='bold')
plt.xticks([1,2,3],['Jan', 'Feb', 'Mar'], fontsize=15)
plt.show()

<b> These results reflect that Feb 2013 was a 'down' month in terms of the nubmer of entries and exits. One potential driver could be inclement weather in Feb 2013. <b>

<b> Plot the daily number of closed stations and number of stations that were not operating at full capacity in Q1 2013.
    
 We will approach this by first denoting 'closed' as stations where Busyness is less than 10 entries/exits.  <b>

In [ ]:
Sum_Busyness_GroupDateStation = df_q1_2013.groupby(['DATE','STATION_x']).sum().BUSYNESS.reset_index()
Sum_Busyness_GroupDateStation['BUSYNESS'] = Sum_Busyness_GroupDateStation['BUSYNESS'].astype(int)

#Calculate sum of busyness by date + station

#if sum(busyness) <= 10 then station is closed. 
Closed_Stations = Sum_Busyness_GroupDateStation[Sum_Busyness_GroupDateStation['BUSYNESS'] <= 10].groupby('DATE')['STATION_x'].agg('count')
Closed_Stations.plot()

plt.title('Number of Closed Stations by Date, Q1 2013', fontsize = 15,fontweight='bold')
plt.xlabel('Date', fontsize = 15, fontweight='bold')
plt.ylabel('Number of Closed Stations', fontsize = 15, fontweight='bold')
plt.show()


#Assumption: If Station's Busyness is < 80% of its mean busyness, station is operating at reduced capacity. 



In [ ]:
#Mean Busyness = Mean busyness by station over 2013
MeanBusyness = Sum_Busyness_GroupDateStation.groupby(['STATION_x']).mean().BUSYNESS
Merged_Busyness = pd.merge(Sum_Busyness_GroupDateStation, MeanBusyness, on = ['STATION_x'])
Merged_Busyness

#Reduced Capacity = if station is less than 60% of its annual average and busyness > 10(baseline), then station is reduced capacity
Reduced_Capacity = Merged_Busyness[(Merged_Busyness['BUSYNESS_x'] <= (.6*Merged_Busyness['BUSYNESS_y'])) & (Merged_Busyness['BUSYNESS_x'] > 10)].groupby('DATE')['STATION_x'].agg('count')
Reduced_Capacity.plot()
plt.title('Number of Reduced Capacity Stations by Date, Q1 2013', fontsize = 15,fontweight='bold')
plt.ylabel('Reduced Capacity Stations', fontsize = 15, fontweight='bold')
plt.show()

<b> While the above is a basic way of approximating stations at reduced capacity, there are ways that this assessment could be improved.
    
        1) First - we should account for quarterly seasonality by looking potentially at multiple years of data.
    
        2) Second - If provided with dates/stations not at full capacity, we could use clustering/anomaly detection to find drivers of reduced capacity to better identify these in the future. 
    